## A feasible Polish grid scenario

The scenario I've been using has only 20 wind farms and 50% wind penetration. I suspect this is nowhere near enough wind to influence most lines in the network. The solutions found by my new secular equation solver have very high objective value.

In [1]:
# single proc
include("../src/TemporalInstanton.jl")
using TemporalInstanton

In [1]:
# two worker procs
addprocs(2)
@everywhere include("../src/TemporalInstanton.jl")
@everywhere using TemporalInstanton

In [ ]:
include("../src/mat2tmpinst.jl")
d = mat2tmpinst("case3120sp",200,0.5)

# Thermal model parameters:
d.Tamb = 35.0 # C
d.T0 = 60.0 #46.0 # initial line steady-state temp

d.time_values = 0.0:30.0:300.0 # five minutes in 30-sec steps
d.int_length = 300. # seconds = 5 min

Gp,Dp,Rp = d.G0,d.D0,d.R0
d.G0 = [0.7*Gp;0.7*Gp;0.7*Gp]#;0.7*Gp;0.7*Gp;0.7*Gp]
d.D0 = [Dp;Dp;Dp]#;0.9*Dp;0.9*Dp;0.9*Dp]
d.R0 = [Rp;1.1*Rp;1.2*Rp]#;1.3*Rp;1.4*Rp;1.5*Rp]

@time results = solve_temporal_instanton(d);

	From worker 3:	finished 1900/3693


In [16]:
n = 3120
nr = 20
T = 3
o = process_instanton_results(results,
    n,
    nr,
    T;
    return_as_type=true);

In [17]:
o.score

10-element Array{Float64,1}:
  34.7771
 111.871 
  94.8901
  30.4662
 251.012 
 112.804 
  86.7148
 244.375 
 136.997 
  48.9684

## Whoa, that worked.

Turns out all I needed was more wind farms. Twenty farms is nowhere near enough to excessively heat lines on the Polish grid.

## Polish grid results

I let the computer run for 130 minutes to analyze the Polish grid. That averages to 2.15 seconds per line. I saved the data in `polish.jld`.

In [1]:
include("../src/TemporalInstanton.jl")
using TemporalInstanton, JLD

d = load("polish.jld")
i = d["inputData"]
o = d["outputData"]

Dict{ByteString,Any} with 2 entries:
  "o"       => TemporalInstanton.InstantonOutputData([32.4138,113.677,93.2212,2…
  "results" => [([0.00181108,0.00218249,-0.0233197,-0.0233061,-0.0175401,-0.002…

In [24]:
# lines sorted by objective value
sortperm(o.score)

3682-element Array{Int64,1}:
 2630
 2769
 2204
  793
   27
 2177
  237
  105
  162
 2317
  762
   40
   42
    ⋮
 3624
 3625
 3628
 3629
 3643
 3644
 3647
 3650
 3652
 3658
 3662
 3667

In [34]:
length(o.score) + length(find(i.res.==0))

3693

In [31]:
o.a

3682-element Array{Array{Array{Float64,1},1},1}:
 [[0.0,0.0297562,-0.225266,-0.22533,-0.259709,-0.0628692,-0.0629339,-0.110292,-0.110098,-0.186018  …  -0.124299,-0.261172,0.00599203,-0.103219,0.0276778,-0.0878744,-0.559349,0.0486005,-0.108844,-0.396291],[0.0,0.0311953,-0.260439,-0.260509,-0.292346,-0.0704284,-0.0704902,-0.136528,-0.136262,-0.215446  …  -0.135111,-0.282467,0.000684082,-0.106821,0.0252719,-0.113129,-0.601107,0.048108,-0.120664,-0.440935],[0.0,0.0348132,-0.468234,-0.468355,-0.481142,-0.095259,-0.0952648,-0.311744,-0.311083,-0.413168  …  -0.0500052,-0.294097,0.0390281,-0.0131431,0.0766821,-0.0831325,-0.628454,0.110954,-0.121067,-0.520714]]           
 [[0.0,0.0298431,-0.229756,-0.229815,-0.264504,-0.0639982,-0.0640639,-0.110653,-0.110441,-0.186091  …  -0.135829,-0.263714,0.00044556,-0.113921,0.0183706,-0.0966569,-0.56236,0.038998,-0.110361,-0.401944],[0.0,0.0306274,-0.229558,-0.229615,-0.263922,-0.0645687,-0.0646357,-0.11053,-0.110309,-0.18845  …  -0.129781,-0.275206,-0.00